## Import Data from EVSE

In [ ]:
import requests
import collections
collections.Iterable = collections.abc.Iterable
import pandas as pd
from pandas import json_normalize

url = "https://data.geo.admin.ch/ch.bfe.ladestellen-elektromobilitaet/data/ch.bfe.ladestellen-elektromobilitaet.json"

response = requests.get(url)

response_json = response.json()

df = json_normalize(response_json["EVSEData"])

df

In [ ]:
print(df.iloc[0])

df_data_record = pd.json_normalize(data=response_json["EVSEData"], record_path='EVSEDataRecord')
df_data_record

In [10]:
country_lat_lon = {
    "country": [],
    "lat": [],
    "lon": []
}   
df_lat_lon = pd.DataFrame(country_lat_lon)

for i in range(0, len(df_data_record)):
    if (df_data_record.iloc[i]['Address.Country'] == "CHE"):
        lat_lon_list = df_data_record.iloc[i]['GeoCoordinates.Google'].split()
        lat_lon_list.insert(0, df_data_record.iloc[i]['Address.Country'])
        df_lat_lon.loc[len(df_lat_lon)] = lat_lon_list

converted_coords = {
    "country": [],
    "lat": [],
    "lon": []
}  

convert_dict = {
    "country": str,
    "lat": float,
    "lon": float
}

df_lat_lon = df_lat_lon.astype(convert_dict)

df_corrected_coords = pd.DataFrame(converted_coords)

df_corrected_coords = df_corrected_coords.astype(convert_dict)


for index, row in df_lat_lon.iterrows():
    if df_lat_lon.iloc[index]['lat'] > 45.818218 and df_lat_lon.iloc[index]['lat'] < 47.808277 and df_lat_lon.iloc[index]['lon'] > 5.956118 and df_lat_lon.iloc[index]['lon'] < 10.492499:
        df_corrected_coords = pd.concat([df_corrected_coords, pd.DataFrame([row])], ignore_index=True)

print(df_corrected_coords.dtypes)
df_corrected_coords

country     object
lat        float64
lon        float64
dtype: object


,country,lat,lon
0,CHE,47.418408,8.373644
1,CHE,47.028107,8.631626
2,CHE,47.418408,8.373644
3,CHE,47.028107,8.631626
4,CHE,47.028107,8.631626
...,...,...,...
12495,CHE,47.519055,8.718913
12496,CHE,47.355510,8.325744
12497,CHE,47.479983,9.531401
12498,CHE,47.355510,8.325744


In [ ]:
import os
import mysql.connector

def save_data_in_db():
    # Set your environment variables or provide them directly in the code
    host = os.environ.get("MYSQL_HOST", "localhost")
    port = os.environ.get("MYSQL_PORT", "3306")
    dbname = os.environ.get("MYSQL_DBNAME", "test")
    user = os.environ.get("MYSQL_USER", "root")
    password = os.environ.get("MYSQL_PASSWORD", "")

    table_name = "charger_data_geocoded"

    try:
        conn = mysql.connector.connect(
            host=host,
            port=port,
            user=user,
            password=password,
            database=dbname
        )
        if conn.is_connected():
            # Create a cursor
            cursor = conn.cursor()

            for index, row in df_corrected_coords.iterrows():
                query = f"INSERT INTO {table_name} (country, lat, lon) VALUES (%s, %s, %s)"
                values = (row['country'], row['lat'], row['lon'])
                cursor.execute(query, values)

            conn.commit()
            cursor.close()
            
            """ # Define your SELECT query
            query = "SELECT * FROM test_table"
            
            # Execute the query
            cursor.execute(query)
            
            # Fetch the results into a Pandas DataFrame
            result = cursor.fetchall()
            df = pd.DataFrame(result, columns=cursor.column_names)
            
            print("Query result:")
            print(df) """
        else:
            print("Connection to MySQL database failed.")
    except mysql.connector.Error as err:
        print(f"Error: {err}")

    finally: 
        if conn is not None:
            conn.close()


In [ ]:
# Save the dataframe in mySQL DB
save_data_in_db()

#### Todo: Check if it works with gmaps. Else find another solution

In [ ]:
""" # Initialisierung der Map
m = folium.Map(location=[47.44, 8.65], zoom_start=10)

# Add lat/lon of addresses
df_sub = df_lat_lon.dropna()
for i in range(0, 300):
    folium.Marker(location=(df_sub.iloc[i]['lat'], 
                            df_sub.iloc[i]['lon'])).add_to(m)

# Layer control
folium.LayerControl().add_to(m)

# Plot map
m """